In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 11:21:05.155607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 11:21:05.258224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 11:21:05.258242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 11:21:05.279659: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 11:21:05.880201: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

2022-11-07 11:21:10.297981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 11:21:10.298653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:21:10.298720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 11:21:10.298778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 11:21:10.298824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 0s 414ms/step - loss: 2.1497 - auc: 0.6520
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.1040 - auc: 0.8258
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.9915 - auc: 0.8462
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7612 - auc: 0.8683
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5925 - auc: 0.8760
Epoch 6/100
1/1 [==============================] - 0s 40ms/step - loss: 1.4978 - auc: 0.8798
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4336 - auc: 0.8857
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3168 - auc: 0.9227
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2191 - auc: 0.9482
Epoch 10/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1146 - auc: 0.9703
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 36ms/step - loss: 0.1937 - auc: 0.9944
Epoch 89/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1811 - auc: 0.9966
Epoch 90/100
1/1 [==============================] - 0s 38ms/step - loss: 0.1884 - auc: 0.9952
Epoch 91/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1827 - auc: 0.9951
Epoch 92/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1821 - auc: 0.9959
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1825 - auc: 0.9959
Epoch 94/100
1/1 [==============================] - 0s 38ms/step - loss: 0.1739 - auc: 0.9970
Epoch 95/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1785 - auc: 0.9964
Epoch 96/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1868 - auc: 0.9948
Epoch 97/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1825 - auc: 0.9954
Epoch 98/100
1/1 [==============================] - 0s 34ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 371ms/step - loss: 1.9216 - auc: 0.7680
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7435 - auc: 0.7684
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6121 - auc: 0.8558
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5049 - auc: 0.8903
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3689 - auc: 0.9149
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2693 - auc: 0.9278
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1869 - auc: 0.9406
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1189 - auc: 0.9516
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0451 - auc: 0.9641
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9711 - auc: 0.9748
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.1921 - auc: 0.9958
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2070 - auc: 0.9933
Epoch 90/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1978 - auc: 0.9945
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1993 - auc: 0.9942
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1990 - auc: 0.9941
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2041 - auc: 0.9939
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1895 - auc: 0.9957
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1940 - auc: 0.9953
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1982 - auc: 0.9948
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1956 - auc: 0.9953
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 2.5713 - auc: 0.3571
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7863 - auc: 0.7999
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8267 - auc: 0.8595
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8743 - auc: 0.8819
Epoch 5/100
1/1 [==============================] - 0s 39ms/step - loss: 1.8908 - auc: 0.8867
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7771 - auc: 0.9026
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6225 - auc: 0.9119
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4645 - auc: 0.9211
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3138 - auc: 0.9326
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2133 - auc: 0.9413
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.1958 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1944 - auc: 0.9953
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1980 - auc: 0.9954
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1991 - auc: 0.9943
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1920 - auc: 0.9958
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1898 - auc: 0.9955
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1917 - auc: 0.9955
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1988 - auc: 0.9943
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1929 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1900 - auc: 0.9958
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 2.5622 - auc: 0.3615
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7016 - auc: 0.8172
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7133 - auc: 0.8809
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8140 - auc: 0.8891
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8406 - auc: 0.8953
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7550 - auc: 0.9026
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6185 - auc: 0.9141
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4682 - auc: 0.9243
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2892 - auc: 0.9392
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2028 - auc: 0.9436
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.1965 - auc: 0.9953
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1923 - auc: 0.9957
Epoch 90/100
1/1 [==============================] - 0s 21ms/step - loss: 0.1924 - auc: 0.9959
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2050 - auc: 0.9923
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1873 - auc: 0.9961
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1964 - auc: 0.9943
Epoch 94/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1942 - auc: 0.9946
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1929 - auc: 0.9950
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1888 - auc: 0.9957
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2003 - auc: 0.9930
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 355ms/step - loss: 3.1516 - auc: 0.2398
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8482 - auc: 0.7189
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7488 - auc: 0.8573
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8815 - auc: 0.8755
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9468 - auc: 0.8850
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9460 - auc: 0.8860
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.8648 - auc: 0.8997
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7514 - auc: 0.9039
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 1.5857 - auc: 0.9185
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4285 - auc: 0.9263
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2040 - auc: 0.9947
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2037 - auc: 0.9948
Epoch 90/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2115 - auc: 0.9941
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1998 - auc: 0.9951
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1996 - auc: 0.9950
Epoch 93/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2096 - auc: 0.9936
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2073 - auc: 0.9941
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1987 - auc: 0.9949
Epoch 96/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2003 - auc: 0.9946
Epoch 97/100
1/1 [==============================] - 0s 43ms/step - loss: 0.1949 - auc: 0.9958
Epoch 98/100
1/1 [==============================] - 0s 49ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MHFP6','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 524ms/step - loss: 2.5794 - auc: 0.3588
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.8450 - auc: 0.7792
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8654 - auc: 0.8489
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9346 - auc: 0.8666
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9497 - auc: 0.8723
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8080 - auc: 0.8852
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6459 - auc: 0.8998
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4915 - auc: 0.9070
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3461 - auc: 0.9158
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2540 - auc: 0.9234
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2264 - auc: 0.9926
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2224 - auc: 0.9925
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2242 - auc: 0.9923
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2250 - auc: 0.9925
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2132 - auc: 0.9944
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2158 - auc: 0.9938
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2262 - auc: 0.9919
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2205 - auc: 0.9927
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2228 - auc: 0.9927
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2167 - auc: 0.9929
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [9]:
m='remove_MHFP6'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 443ms/step - loss: 2.1122 - auc: 0.5798
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.9262 - auc: 0.8076
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9260 - auc: 0.8418
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.7716 - auc: 0.8653
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6427 - auc: 0.8716
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4657 - auc: 0.8841
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 1.3689 - auc: 0.8901
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3442 - auc: 0.8768
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2899 - auc: 0.8857
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1669 - auc: 0.9279
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2089 - auc: 0.9934
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2165 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2146 - auc: 0.9921
Epoch 91/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2114 - auc: 0.9923
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2182 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2084 - auc: 0.9931
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2174 - auc: 0.9904
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2111 - auc: 0.9922
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2035 - auc: 0.9933
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2221 - auc: 0.9909
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [10]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 348ms/step - loss: 5.0548 - auc: 0.2320
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.8674 - auc: 0.3296
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7468 - auc: 0.7876
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6285 - auc: 0.8974
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7948 - auc: 0.9041
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9851 - auc: 0.8917
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 2.1062 - auc: 0.8845
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 2.1289 - auc: 0.8827
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 2.0991 - auc: 0.8796
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 2.0518 - auc: 0.8858
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2537 - auc: 0.9938
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2456 - auc: 0.9950
Epoch 90/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2435 - auc: 0.9948
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2512 - auc: 0.9930
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2455 - auc: 0.9941
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2399 - auc: 0.9950
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2439 - auc: 0.9935
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2341 - auc: 0.9948
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2353 - auc: 0.9943
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2318 - auc: 0.9947
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [11]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 527ms/step - loss: 2.3053 - auc: 0.4457
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7514 - auc: 0.8228
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7927 - auc: 0.8678
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8437 - auc: 0.8791
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7679 - auc: 0.8905
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6586 - auc: 0.8929
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4708 - auc: 0.9118
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3130 - auc: 0.9255
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2429 - auc: 0.9260
Epoch 10/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1893 - auc: 0.9301
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1982 - auc: 0.9952
Epoch 89/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2100 - auc: 0.9937
Epoch 90/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2079 - auc: 0.9942
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2034 - auc: 0.9940
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2056 - auc: 0.9948
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2058 - auc: 0.9940
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2010 - auc: 0.9953
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2046 - auc: 0.9947
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2099 - auc: 0.9931
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2008 - auc: 0.9947
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [12]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 365ms/step - loss: 2.0561 - auc: 0.6932
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9678 - auc: 0.8166
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7594 - auc: 0.8361
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5349 - auc: 0.8619
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4673 - auc: 0.8544
Epoch 6/100
1/1 [==============================] - 0s 37ms/step - loss: 1.3496 - auc: 0.8952
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2143 - auc: 0.9407
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1309 - auc: 0.9591
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0773 - auc: 0.9669
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0275 - auc: 0.9714
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.1851 - auc: 0.9940
Epoch 89/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1858 - auc: 0.9955
Epoch 90/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1763 - auc: 0.9964
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1845 - auc: 0.9952
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1810 - auc: 0.9956
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1818 - auc: 0.9950
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1826 - auc: 0.9956
Epoch 95/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1807 - auc: 0.9956
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1810 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1813 - auc: 0.9953
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [13]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MHFP6','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 397ms/step - loss: 2.9097 - auc: 0.2831
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8135 - auc: 0.7652
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8009 - auc: 0.8529
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9279 - auc: 0.8680
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9612 - auc: 0.8745
Epoch 6/100
1/1 [==============================] - 0s 42ms/step - loss: 1.8972 - auc: 0.8863
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7741 - auc: 0.8951
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6047 - auc: 0.9083
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4831 - auc: 0.9142
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3109 - auc: 0.9279
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.2112 - auc: 0.9954
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2103 - auc: 0.9952
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2136 - auc: 0.9942
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2092 - auc: 0.9949
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2033 - auc: 0.9956
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2034 - auc: 0.9954
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2046 - auc: 0.9953
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2060 - auc: 0.9949
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2008 - auc: 0.9961
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1965 - auc: 0.9960
Epoch 98/100
1/1 [==============================] - 0s 36ms/step - loss: 

In [14]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('10feature_remove_one3.csv',index=False)
print(de)

                  model       auc
0             all_model  0.944274
1     remove_AtomPairFP  0.930080
2   remove_Connectivity  0.940408
3         remove_Charge  0.943688
4          remove_Kappa  0.944254
5           remove_MAP4  0.941573
6          remove_MHFP6  0.945274
7            remove_MOE  0.942575
8       remove_MorganFP  0.943210
9       remove_Topology  0.942546
10     remove_TorsionFP  0.945005
